In [153]:
from datetime import datetime, timedelta
import os
import re
import pandas as pd

from calculate_flux import get_fluency_flux
from calculate_flux import read_count_file
from calculate_flux import calculate_open_periods


# Directory path
directory = "./"
distance_factor_file = "distance_file.txt"
allowed_devices = ['bruno1', 'bruno2']
shutter_file = "shutter.txt"

# Time window for each run
time_unit_offset = timedelta(hours=0.1)


In [154]:
# Get all countlog files in the directory
countlog_files = [filename for filename in os.listdir(directory) if filename.startswith("countlog")]

# Sort the countlog files in chronological order
countlog_files.sort(key=lambda x: datetime.strptime(re.search(r"\d{4}-\d{2}-\d{2}", x).group(), "%Y-%m-%d"))

# Initialize an empty dataframe
neutron_count_df = pd.DataFrame()

# Loop through the countlog files in chronological order
for filename in countlog_files:
    # Get the full file path
    file_path = os.path.join(directory, filename)

    print(filename)
    # Read the count file and append it to the dataframe
    neutron_count = read_count_file(file_path)
    neutron_count_df = pd.concat([neutron_count_df, pd.DataFrame(neutron_count)]).reset_index(drop=True)


neutron_count_df = neutron_count_df.rename(columns={0: "Timestamp", 1: "Fission Count"})

# Load all distances before continue
distance_data = pd.read_csv(distance_factor_file, sep="\t")

countlog-2023-12-04.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-05.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-06.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-07.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-08.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-09.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-10.txt
Ignoring line (malformed):Data From ChipIR



In [155]:
#Filter out the data that is not in the distance file for each device
devices_dataframe = {}

for device in distance_data["board"].unique():
    if device in allowed_devices:
        distance_line = distance_data[distance_data["board"] == device]
        start_line = distance_line["start"].iloc[0]
        end_line = distance_line["end"].iloc[0]

        start_dt = pd.to_datetime(start_line, format='%d/%m/%Y %H:%M:%S')
        end_dt = pd.to_datetime(end_line, format='%d/%m/%Y %H:%M:%S')
        
        filtered_df = neutron_count_df[
            (neutron_count_df["Timestamp"] >= start_dt) &
            (neutron_count_df["Timestamp"] <= end_dt)
        ]
        
        devices_dataframe[device] = filtered_df

In [156]:
#Filter dataframes to remove open shutter times

# def display_periods(periods):
#     """ Display the open periods """
#     total_duration = timedelta()
#     for start, end in periods:
#         total_duration += end - start
        
#     total_duration_hours = total_duration.total_seconds() / 3600
#     print(f"Shutter open for a total time of {total_duration_hours} hours")

# periods = calculate_open_periods(shutter_file)
# display_periods(periods)

# shutter_dataframe = {}

# for device, df in devices_dataframe.items():
#     filtered_df = pd.DataFrame()
#     for start, end in periods:
#         filtered_df = pd.concat([filtered_df, df[(df["Timestamp"] >= start) & (df["Timestamp"] <= end)]])
#     shutter_dataframe[device] = filtered_df


In [157]:

device_dict = {}

for device in allowed_devices:
    device_flunce_pd = pd.DataFrame(columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])
    
    df = devices_dataframe[device]

    distance_line = distance_data[
        (distance_data["board"].str.contains(device))
    ]
    facility_factor = float(distance_line["facility_factor"])
    distance_attenuation = float(distance_line["Distance attenuation"])

    start_dt = df["Timestamp"].iloc[0]

    while start_dt <= df["Timestamp"].iloc[-1]:
        end_dt = start_dt + time_unit_offset
        
        if end_dt > df["Timestamp"].iloc[-1]:
            break

        mistake_time = pd.to_datetime("2023-12-08 10:23:00")
        if start_dt <= mistake_time and end_dt >= mistake_time:
            # print("aaaaaa",start_dt, end_dt,mistake_time)
            start_dt = start_dt + time_unit_offset
            continue

        neutron_count = df[(df["Timestamp"] >= start_dt) & (df["Timestamp"] <= end_dt)].values
        if len(neutron_count) == 0:
            start_dt = start_dt + time_unit_offset
            continue
        

        flux, time_beam_off = get_fluency_flux(
            start_dt=start_dt,
            end_dt=end_dt,
            neutron_count=neutron_count,
            facility_factor=facility_factor,
            distance_attenuation=distance_attenuation,
        )
        
        fluence = flux * time_unit_offset.total_seconds()  
        data = [
            [start_dt, flux, time_beam_off, flux * time_unit_offset.total_seconds()]
        ]
        device_flunce_pd = pd.concat([device_flunce_pd, pd.DataFrame(data, columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])], ignore_index=True)
        # print(start_dt, flux, time_beam_off, fluence)

        start_dt = end_dt
    
    device_dict[device] = device_flunce_pd



In [158]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Filter dataframes to remove open shutter times

def display_periods(periods):
    """ Display the open periods """
    total_duration = timedelta()
    for start, end in periods:
        total_duration += end - start
        
    total_duration_hours = total_duration.total_seconds() / 3600
    print(f"Shutter open for a total time of {total_duration_hours} hours")

periods = calculate_open_periods(shutter_file)
display_periods(periods)

# FILEPATH: /home/beforlin/projects/radhelper-embedded/post_processing/flux.ipynb
device_dict["bruno1"] = device_dict["bruno1"][device_dict["bruno1"]["Flux"] != 0]
total_time_beam_off = device_dict["bruno1"]["Time Beam Off"].sum()
num_time_windows = device_dict["bruno1"].shape[0] * time_unit_offset.total_seconds() / 3600
total_fluence = device_dict["bruno1"]["Fluence"].sum()
print(f"{round(total_time_beam_off/3600, 2)} hours of beam off out of {num_time_windows} hours") # in hours
print(f"Total Fluence = {total_fluence:.2e}")
device_dict["bruno1"]



Shutter open for a total time of 64.46666666666667 hours
3.72 hours of beam off out of 54.3 hours
Total Fluence = 8.99e+11


Start Date     Flux  Time Beam Off  Fluence
167 2023-12-06 13:42:00.403 3.52e+05       3.04e+02 1.27e+08
168 2023-12-06 13:48:00.403 3.75e+06       0.00e+00 1.35e+09
169 2023-12-06 13:54:00.403 5.01e+06       0.00e+00 1.81e+09
170 2023-12-06 14:00:00.403 4.93e+06       0.00e+00 1.78e+09
171 2023-12-06 14:06:00.403 4.98e+06       0.00e+00 1.79e+09
172 2023-12-06 14:12:00.403 4.99e+06       0.00e+00 1.80e+09
173 2023-12-06 14:18:00.403 4.97e+06       0.00e+00 1.79e+09
174 2023-12-06 14:24:00.403 4.97e+06       0.00e+00 1.79e+09
175 2023-12-06 14:30:00.403 4.98e+06       0.00e+00 1.79e+09
176 2023-12-06 14:36:00.403 4.97e+06       0.00e+00 1.79e+09
177 2023-12-06 14:42:00.403 4.98e+06       0.00e+00 1.79e+09
178 2023-12-06 14:48:00.403 5.00e+06       0.00e+00 1.80e+09
182 2023-12-06 15:12:00.403 3.51e+03       3.51e+02 1.26e+06
184 2023-12-06 15:24:00.403 1.93e+05       1.30e+02 6.95e+07
185 2023-12-06 15:30:00.403 3.70e+06       0.00e+00 1.33e+09
186 2023-12-06 15:36:00.403 4.93e+06       0.00e+00 1.77e+09
187 2023-12-06 15:42:00.403 4.94e+06       0.00e+00 1.78e+09
188 2023-12-06 15:48:00.403 4.90e+06       0.00e+00 1.76e+09
189 2023-12-06 15:54:00.403 4.91e+06       0.00e+00 1.77e+09
190 2023-12-06 16:00:00.403 4.91e+06       0.00e+00 1.77e+09
191 2023-12-06 16:06:00.403 4.87e+06       0.00e+00 1.75e+09
192 2023-12-06 16:12:00.403 4.85e+06       0.00e+00 1.75e+09
193 2023-12-06 16:18:00.403 4.82e+06       0.00e+00 1.73e+09
194 2023-12-06 16:24:00.403 4.81e+06       0.00e+00 1.73e+09
195 2023-12-06 16:30:00.403 4.75e+06       3.10e+00 1.71e+09
196 2023-12-06 16:36:00.403 4.98e+06       0.00e+00 1.79e+09
197 2023-12-06 16:42:00.403 5.00e+06       0.00e+00 1.80e+09
198 2023-12-06 16:48:00.403 5.02e+06       0.00e+00 1.81e+09
199 2023-12-06 16:54:00.403 5.01e+06       0.00e+00 1.80e+09
200 2023-12-06 17:00:00.403 2.01e+06       2.08e+02 7.24e+08
211 2023-12-06 18:06:00.403 4.28e+06       4.66e+01 1.54e+09
212 2023-12-06 18:12:00.403 4.98e+06       0.00e+00 1.79e+09
213 2023-12-06 18:18:00.403 4.97e+06       0.00e+00 1.79e+09
214 2023-12-06 18:24:00.403 4.98e+06       0.00e+00 1.79e+09
215 2023-12-06 18:30:00.403 4.98e+06       0.00e+00 1.79e+09
216 2023-12-06 18:36:00.403 4.98e+06       0.00e+00 1.79e+09
217 2023-12-06 18:42:00.403 4.95e+06       0.00e+00 1.78e+09
218 2023-12-06 18:48:00.403 4.99e+06       0.00e+00 1.80e+09
219 2023-12-06 18:54:00.403 1.24e+06       2.64e+02 4.46e+08
255 2023-12-06 22:30:00.403 4.44e+06       3.10e+01 1.60e+09
256 2023-12-06 22:36:00.403 4.97e+06       0.00e+00 1.79e+09
257 2023-12-06 22:42:00.403 4.96e+06       0.00e+00 1.79e+09
258 2023-12-06 22:48:00.403 4.96e+06       0.00e+00 1.78e+09
259 2023-12-06 22:54:00.403 3.44e+06       1.06e+02 1.24e+09
260 2023-12-06 23:00:00.403 1.94e+06       2.11e+02 6.99e+08
261 2023-12-06 23:06:00.403 7.99e+05       2.95e+02 2.87e+08
262 2023-12-06 23:12:00.403 2.72e+06       1.55e+02 9.79e+08
263 2023-12-06 23:18:00.403 4.92e+06       0.00e+00 1.77e+09
264 2023-12-06 23:24:00.403 4.95e+06       0.00e+00 1.78e+09
265 2023-12-06 23:30:00.403 4.92e+06       0.00e+00 1.77e+09
266 2023-12-06 23:36:00.403 3.40e+06       4.35e+01 1.22e+09
267 2023-12-06 23:42:00.403 3.93e+06       5.59e+01 1.41e+09
268 2023-12-06 23:48:00.403 4.68e+06       0.00e+00 1.69e+09
269 2023-12-06 23:54:00.403 2.79e+05       3.35e+02 1.00e+08
271 2023-12-07 00:06:00.403 4.93e+05       3.10e+02 1.78e+08
275 2023-12-07 00:30:00.403 1.61e+06       1.66e+02 5.81e+08
276 2023-12-07 00:36:00.403 4.87e+06       0.00e+00 1.75e+09
277 2023-12-07 00:42:00.403 5.04e+06       0.00e+00 1.82e+09
278 2023-12-07 00:48:00.403 5.10e+06       0.00e+00 1.84e+09
279 2023-12-07 00:54:00.403 5.10e+06       0.00e+00 1.84e+09
280 2023-12-07 01:00:00.403 5.09e+06       0.00e+00 1.83e+09
281 2023-12-07 01:06:00.403 5.07e+06       0.00e+00 1.82e+09
282 2023-12-07 01:12:00.403 5.11e+06       0.00e+00 1.84e+09
283 2023-12-07 01:18:00.403 5.11e+06       0.00e+00 1.84e+09
284 2023-12-07 01:24:00.403 5.07e+06       0.00e+00 